In [77]:
import joblib
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
import requests
import json

# Load Posenet

In [10]:
interpreter = tf.lite.Interpreter("C:/Users/hjhhi/Desktop/Codes/lite-model_movenet_singlepose_lightning_tflite_float16_4.tflite")
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
interpreter.allocate_tensors()
output = interpreter.get_output_details()[0]  # Model has single output.
input = interpreter.get_input_details()[0]  # Model has single input.
interpreter.invoke()

In [27]:
def image_preprocess(path):
    image = cv2.imread(path)
    image= image.astype("uint8")
    image = cv2.resize(image, (192, 192), interpolation=cv2.INTER_NEAREST)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image= np.resize(image, (1, 192, 192, 3))
    return image

아래쪽의 이미지는 모비우스에서 가져오고 base64 디코딩 과정을 거쳐야함

In [28]:
image = image_preprocess("C:/Users/hjhhi/Desktop/Codes/51.jpg")

interpreter.set_tensor(input['index'], image)
interpreter.invoke()
result = interpreter.get_tensor(output_details[0]['index'])
result = result.reshape(51)

# Load image from Mobius

In [81]:
url = "http://203.253.128.177:7579/Mobius/SW_Hackaton/test/4-20220916095142457"



payload=""
headers = {
'Accept': 'application/json',
'X-M2M-RI': '12345',
'X-M2M-Origin': 'S',
'Content-Type': 'application/json'
}

response = requests.request("GET", url, headers=headers, data=payload)
json.loads(response.text)["m2m:cin"]["con"]

'10000'

# Load image_classifier

In [29]:
image_classifier = joblib.load("./image_model.pkl")

In [34]:
image_proba = image_classifier.predict_proba([result])

In [35]:
image_proba

array([[0.01758501, 0.00950702, 0.57819063, 0.12256325, 0.05857313,
        0.08835219, 0.12522878]])

# Load gyro_classifier 

In [82]:
gyro_classifier = joblib.load("./gyro_model.pkl")

In [87]:
gyro_proba = gyro_classifier.predict_proba([[-0.0122173 , -0.05497787,  0.01710423, -2.6001031 , -9.3182335 ,1.0462662]])

C:\Users\hjhhi\Anaconda3\envs\raspberry\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [88]:
gyro_proba

array([[0.        , 0.04970075, 0.06602758, 0.10212088, 0.73870548,
        0.0434453 , 0.        ]])

In [89]:
image_proba + gyro_proba

array([[0.01758501, 0.05920778, 0.64421821, 0.22468413, 0.79727861,
        0.13179749, 0.12522878]])

In [91]:
0.00950702 +  0.04970075

0.05920777

# Send final prediction to Mobius 

In [41]:
Final_pred=10000

In [43]:
import requests
import json

url = "http://203.253.128.177:7579/Mobius/SW_Hackaton/test"

data = {
    "m2m:cin": {
        "con": Final_pred
    }
}

headers = {
  'Accept': 'application/json',
  'X-M2M-RI': '12345',
  'X-M2M-Origin': 'SOrigin',
  'Content-Type': 'application/json;ty=4'
}



response = requests.request("POST", url, headers=headers, data=json.dumps(data))

print(response.text)


{"m2m:cin":{"rn":"4-20220916095142457","ty":4,"pi":"3-20220915105957489894","ri":"4-20220916095142458460","ct":"20220916T095142","lt":"20220916T095142","st":14,"et":"20240916T095142","cs":5,"con":"10000","cr":"SOrigin"}}
